In [1]:
import json
import os
import re
import xml.etree.ElementTree as et
path="your_dir/VOC3000/Annotations/3000_"
imgpath="your_dir/VOC3000/JPEGImages/3000_"
num=3147
def get_namespace(element):  # 获取节点的namespace
    m = re.match('\{.*\}', element.tag)
    return m.group(0) if m else ''
data_set=[]
monum=0
mrnum=0
map_obj={}
map_rel={}
for i in range(num):
    if i % 10 ==0:
        print(i)
    strnum=str(i).zfill(6)
    tpath=path+strnum+".xml"
    if not os.path.exists(tpath):
        continue
    tree=et.parse(tpath)
    
    root=tree.getroot()
    ns=get_namespace(root)
    #print(ns)
    #print(root.tag)
    img_data=[]
    obj=[]
    relation_dict=[]
    img_size=[]
    for child in root:
        #print(child.tag)
        
        if child.tag=="object":
            tmp=[]
            for cd in child:
                if cd.tag=="name":
                    #print(cd.text)
                    if cd.text not in map_obj.keys():
                        if cd.text.find("_")==-1:
                            map_obj[cd.text]=monum
                            monum+=1
                    tmp.append(cd.text)
                if cd.tag=="object_id":
                    tmp.append(cd.text)
                if cd.tag=="bndbox":
                    for n in cd:
                        tmp.append(n.text)
            #print(tmp)
            obj.append(tmp)
            continue
        if child.tag=="size":
            for cd in child:
                img_size.append(cd.text)
    obj_data=[]
    if len(obj)==3:
        flag=1
        relnum=0
        for h in range(len(obj)):
            if obj[h][0].find("_")!=-1:
                relnum+=1
                flag=0
                tmp=obj[h][0].split("_")
                a1=0
                a2=len(tmp)-1
                #这一步是因为数据集奇葩，15th person lying in beach object里面没有beach，吐了
                if tmp[a1] in map_obj.keys():
                    relation_dict.append(tmp[a1])
                else:
                    flag=1
                rela=tmp[a1+1]
                for o in range(a1+2,a2):
                    rela=rela+"_"+tmp[o]
                if rela not in map_rel.keys():
                    map_rel[rela]=mrnum
                    mrnum+=1
                relation_dict.append(rela)
                if tmp[a2] in map_obj.keys():
                    relation_dict.append(tmp[a2])
                else:
                    flag=1
            else:
                obj_data.append(obj[h])
        if flag==1 or relnum>1:
            continue
        img_data.append(obj_data)
        img_data.append(relation_dict)
        img_data.append(img_size)
        img_data.append(i)
        data_set.append(img_data)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

In [2]:
def creatsift(img):
    #gray=img
    gray=cv.cvtColor(img,cv2.COLOR_BGR2GRAY)
    sift = cv.xfeatures2d.SIFT_create(200)
    kp,des=sift.detectAndCompute(gray,None)
    return des



def build_center(img_mat):
    print("start bulid center")
    features = np.float32([]).reshape(0,128)#存放训练集图片的特征
    for im in range(len(img_mat)):
        img_f = creatsift(img_mat[im])
        #特征点加入训练数据
        try:
            features=np.append(features,img_f,axis=0)
        except:
            print(im)
    #训练集的词袋
    return features

def learnVocabulary(feature,num_class):
    print("Voca")
    wordCnt=num_class
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 20, 0.1)
    flags = cv.KMEANS_RANDOM_CENTERS
    #得到k-means的中心点
    compactness, labels, centers = cv2.kmeans(feature, wordCnt, None,criteria, 20, flags)
    print("voce done")
    return centers

#计算特征向量
def calcFeatVec(features, centers):
    featVec = np.zeros((1, fea_num))
    if features is None:
        return featVec
    for i in range(0, features.shape[0]):
        #第i个特征点
        fi = features[i]
        diffMat = fi - centers
        #axis=1按行求和，即求特征到每个中心点的距离
        sqSum = (diffMat**2).sum(axis=1)
        dist = sqSum**0.5
        #升序排序
        sortedIndices = dist.argsort()
        #取出最小的距离，即找到最近的中心点
        idx = sortedIndices[0]
        #该中心点对应+1
        featVec[0][idx] += 1
    return featVec


def cal_vec(img_mat,centers):
    print("cal_vec")
    labels = []
    data_vec = np.float32([]).reshape(0,fea_num)
    count=1
    for im in range(len(img_mat)):
        #获取图片sift特征点
        img_f = creatsift(img_mat[im])
        img_vec = calcFeatVec(img_f, centers)
        try:
            data_vec=np.append(data_vec,img_vec,axis=0)
        except:
            img_vec=np.zeros((1,fea_num))
            data_vec=np.append(data_vec,img_vec,axis=0)
        count+=1
    print("cal done")
    return data_vec

fea_num=529
import cv2 as cv 
import cv2
def get_vec(img_mat):
    
    features=build_center(img_mat)
    centers=learnVocabulary(features,fea_num)
    data_feature=cal_vec(img_mat,centers)
    return data_feature

In [3]:
import cv2
import math
from sklearn.model_selection import GridSearchCV, train_test_split
from skimage import feature as ft
from sklearn import svm
import numpy as np

def get_pix_gray(img_num,loc):
    img=cv2.imread(imgpath+str(img_num).zfill(6)+".jpg",cv2.IMREAD_GRAYSCALE)
    xmin=int(loc[0])
    ymin=int(loc[1])
    xmax=math.ceil(loc[2])
    ymax=math.ceil(loc[3])
    
    #print(img.shape)
    #print(img)
    return img[ymin:ymax,xmin:xmax]
def get_pix_col(img_num,loc):
    img=cv2.imread(imgpath+str(img_num).zfill(6)+".jpg")
    xmin=int(loc[0])
    ymin=int(loc[1])
    xmax=math.ceil(loc[2])
    ymax=math.ceil(loc[3])
    #print(img.shape)
    #print(img)
    img=img[ymin:ymax,xmin:xmax,:]
    #img=img[xmin:xmax,ymin:ymax,:]
    #cv2.imshow("img",img)
    #cv2.imwrite("img"+str(img_num)+".jpg",img)
    #img=cv2.resize(img,(64,64))
    return img
train_s=[]
train_o=[]
train_y=[]
s_DATA=[]
o_DATA=[]
print("len=",len(data_set))
for i in range(len(data_set)):
    print(i)
    Subject=data_set[i][1][0]
    Object=data_set[i][1][2]
    Relation=data_set[i][1][1]
    imgnum=data_set[i][3]
    print(Subject,Relation,Object,imgnum)
    subnum=map_obj[Subject]
    objnum=map_obj[Object]
    relnum=map_rel[Relation]
    img_size=data_set[i][2]
    idx=0
    if data_set[i][0][0][0]==Subject:
        idx=0
    else:
        idx=1
    sub_loc=data_set[i][0][idx][1:]
    obj_loc=data_set[i][0][1-idx][1:]
    for j in range(len(img_size)):
        img_size[j]=float(img_size[j])
    for j in range(len(sub_loc)):
        sub_loc[j]=float(sub_loc[j])
    for j in range(len(obj_loc)):
        obj_loc[j]=float(obj_loc[j])
    imgs=get_pix_col(imgnum,sub_loc)
    imgo=get_pix_col(imgnum,obj_loc)
    '''features=ft.hog(imgs,orientations=9,
                pixels_per_cell=(8,8),
                cells_per_block=(2,2),
                block_norm="L1",
                transform_sqrt=True,
                feature_vector=True,
                visualise=False
               )
    featureo=ft.hog(imgo,orientations=9,
                pixels_per_cell=(8,8),
                cells_per_block=(2,2),
                block_norm="L1",
                transform_sqrt=True,
                feature_vector=True,
                visualise=False
               )'''
    s_DATA.append(imgs)
    o_DATA.append(imgo)
    train_y.append(relnum)
train_s=get_vec(s_DATA)
train_o=get_vec(o_DATA)
xs_train,xs_test,ys_train,ys_test=train_test_split(train_s,train_y,test_size=0.2)
xo_train,xo_test,yo_train,yo_test=train_test_split(train_o,train_y,test_size=0.2)
model1=svm.SVC(decision_function_shape='ovr',kernel='rbf',probability=True)
model2=svm.SVC(decision_function_shape='ovr',kernel='rbf',probability=True)
#print(ys_train)
model1.fit(xs_train,ys_train)
model2.fit(xo_train,yo_train)


len= 819
0
person nextto car 1
1
person sitting_on chair 4
2
person riding bicycle 18
3
person riding horse 19
4
person nextto car 29
5
person drinking bottle 31
6
person jumping_on horse 35
7
person riding horse 38
8
cat lying_on sofa 39
9
person riding bicycle 44
10
person nextto horse 46
11
person riding horse 47
12
person drinking bottle 48
13
person riding horse 49
14
bicycle nextto car 50
15
person jumping_on horse 51
16
person jumping_on horse 53
17
person riding bicycle 58
18
bicycle nextto car 59
19
person jumping_on horse 68
20
person sitting_on chair 70
21
person nextto horse 71
22
person drinking bottle 79
23
person nextto horse 82
24
person sitting_on chair 92
25
person drinking bottle 93
26
person lying_on sofa 94
27
person nextto horse 103
28
person jumping_on horse 108
29
person riding horse 111
30
person nextto car 112
31
person riding horse 120
32
cat lying_on sofa 121
33
person lying_on sofa 122
34
person nextto car 131
35
person sitting_on chair 133
36
person drinki

296
person nextto bicycle 1141
297
person riding horse 1146
298
person riding horse 1147
299
person nextto horse 1152
300
person riding horse 1163
301
person riding bicycle 1164
302
person sitting_on sofa 1165
303
dog lying_on sofa 1166
304
person riding horse 1168
305
person nextto car 1170
306
person sitting_on sofa 1171
307
person jumping_on horse 1173
308
person riding horse 1175
309
cat lying_on sofa 1177
310
person nextto horse 1179
311
person riding horse 1185
312
cat lying_on sofa 1189
313
bicycle nextto car 1190
314
person riding horse 1192
315
person drinking bottle 1193
316
person nextto bicycle 1195
317
person jumping_on horse 1197
318
person sitting_on chair 1198
319
cat lying_on sofa 1204
320
bicycle nextto car 1205
321
cat lying_on sofa 1209
322
person jumping_on horse 1210
323
dog lying_on sofa 1214
324
person riding horse 1217
325
person sitting_on chair 1219
326
cat lying_on sofa 1223
327
bicycle nextto car 1226
328
person jumping_on horse 1233
329
person jumping_on h

571
person riding bicycle 2131
572
bicycle nextto car 2134
573
person jumping_on horse 2141
574
person sitting_on chair 2144
575
person riding horse 2145
576
person drinking bottle 2154
577
dog lying_on sofa 2155
578
bicycle nextto car 2156
579
person nextto horse 2158
580
person drinking bottle 2160
581
person sitting_on chair 2167
582
person nextto car 2172
583
person drinking bottle 2176
584
person nextto bicycle 2185
585
person nextto horse 2186
586
person jumping_on horse 2189
587
person riding horse 2192
588
person jumping_on horse 2198
589
person riding bicycle 2200
590
person sitting_on sofa 2201
591
dog lying_on sofa 2205
592
person nextto car 2208
593
bicycle nextto car 2209
594
bicycle nextto car 2227
595
person riding bicycle 2233
596
person nextto horse 2234
597
person nextto horse 2242
598
person riding horse 2245
599
person jumping_on horse 2247
600
bicycle nextto car 2248
601
person sitting_on sofa 2252
602
person jumping_on horse 2253
603
person lying_on sofa 2257
604


C:\Users\sll82\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\sll82\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [6]:
pr1=model1.predict_proba(xs_test)
p1=model1.predict(xs_test)
#k=len(pr)
#count=0
#for i in range(len(p1)):
    #print(pr1[i],p1[i])
#print(pr[0])



[0.1721142  0.01750581 0.36851947 0.03096335 0.22540389 0.18549328] 3
[0.16580348 0.22473352 0.17079892 0.12151705 0.20932777 0.10781926] 3
[0.05576766 0.51255691 0.0475891  0.2008048  0.03864426 0.14463728] 1
[0.25041129 0.26464796 0.0790035  0.07120209 0.02668637 0.30804878] 6
[0.27399753 0.24132809 0.23241313 0.05103375 0.07540715 0.12582034] 3
[0.24934992 0.19278835 0.12193019 0.11149164 0.160792   0.1636479 ] 0
[0.65976982 0.09074613 0.13349934 0.03525047 0.03976776 0.04096648] 0
[0.32728929 0.23254914 0.11389316 0.06835755 0.02156995 0.2363409 ] 0
[0.08074981 0.07907174 0.50838206 0.0281452  0.0451293  0.25852188] 3
[0.08428372 0.01979688 0.44217736 0.01818621 0.31229783 0.123258  ] 3
[0.44233564 0.17671161 0.20024361 0.04152081 0.0539565  0.08523183] 0
[0.21133938 0.04892619 0.60082119 0.03878183 0.07498525 0.02514616] 3
[0.16774916 0.21451537 0.11349353 0.12841304 0.02658274 0.34924615] 6
[0.37115729 0.16066724 0.24350201 0.05566321 0.06098374 0.10802651] 0
[0.48440466 0.113816

In [7]:
pr2=model2.predict_proba(xo_test)
p2=model2.predict(xo_test)
#for i in range(len(p2)):
    #print(pr2[i],p2[i])

[0.19984542 0.02656448 0.44950958 0.02306552 0.25669401 0.044321  ] 3
[0.14940494 0.09941562 0.44044653 0.01719509 0.25776291 0.03577492] 3
[0.06017936 0.21693908 0.0752715  0.04904272 0.06637285 0.53219449] 6
[0.05072512 0.14313232 0.43361518 0.29181524 0.00841405 0.0722981 ] 3
[0.19283387 0.05909251 0.55435666 0.01382509 0.06566106 0.11423082] 3
[0.54444592 0.04036919 0.25732531 0.01204806 0.12893376 0.01687776] 0
[0.33791927 0.12213673 0.35589565 0.01602686 0.085995   0.08202648] 3
[0.19744556 0.21924177 0.43760973 0.01689173 0.11573037 0.01308083] 3
[0.44018426 0.17089886 0.13539293 0.01324192 0.04078639 0.19949563] 0
[0.08178293 0.06400783 0.04939044 0.01819672 0.03268948 0.75393258] 6
[0.18128642 0.17251797 0.25651654 0.02318284 0.10430444 0.2621918 ] 3
[0.5517606  0.01550377 0.21862337 0.01149095 0.19866567 0.00395563] 0
[0.30940967 0.13583532 0.23232269 0.01476768 0.06038722 0.24727742] 3
[0.59102969 0.07530818 0.17253467 0.01035721 0.06973149 0.08103876] 0
[0.21591579 0.180004

In [8]:
proba_p=[]
posib_map=[0,1,3,4,5,6]
for i in range(len(pr2)):
    tmp=[]
    Max=0
    mi=0
    for j in range(6):
        pso=pr1[i][j]*pr2[i][j]
        if Max<pso:
            mi=j
            Max=pso
    proba_p.append(posib_map[mi])
print(proba_p)
cnt=0
for i in range(len(ys_test)):
    #print(proba_p[i],ys_test[i])
    if proba_p[i]==ys_train[i]:
        cnt+=1
print("correctness=",cnt/len(ys_test))

[3, 3, 1, 1, 3, 0, 0, 0, 3, 6, 0, 3, 6, 0, 0, 6, 6, 0, 0, 0, 3, 0, 4, 3, 6, 3, 0, 3, 5, 3, 0, 3, 6, 1, 3, 3, 0, 3, 3, 3, 3, 5, 3, 3, 0, 0, 6, 0, 3, 0, 6, 3, 6, 0, 1, 0, 1, 3, 0, 3, 0, 6, 3, 3, 0, 3, 0, 6, 0, 0, 1, 0, 3, 3, 3, 6, 3, 3, 0, 4, 3, 6, 0, 5, 0, 3, 3, 3, 6, 3, 0, 3, 3, 3, 0, 3, 3, 0, 3, 5, 3, 6, 6, 0, 3, 6, 0, 3, 3, 6, 3, 0, 6, 3, 0, 3, 0, 3, 3, 3, 0, 0, 1, 4, 0, 3, 3, 0, 3, 6, 3, 3, 0, 0, 3, 6, 3, 5, 3, 1, 3, 3, 0, 0, 0, 0, 6, 3, 3, 6, 3, 3, 6, 3, 3, 3, 3, 3, 6, 5, 3, 0, 6, 6]
3 3
3 0
1 0
1 4
3 5
0 3
0 0
0 3
3 3
6 1
0 0
3 3
6 6
0 3
0 3
6 6
6 6
0 0
0 0
0 0
3 1
0 3
4 1
3 5
6 1
3 0
0 0
3 0
5 3
3 1
0 3
3 1
6 4
1 5
3 3
3 4
0 0
3 0
3 0
3 0
3 3
5 5
3 5
3 6
0 0
0 0
6 0
0 0
3 3
0 0
6 6
3 3
6 5
0 4
1 3
0 0
1 5
3 3
0 0
3 6
0 3
6 4
3 3
3 1
0 1
3 3
0 3
6 6
0 1
0 6
1 4
0 0
3 0
3 4
3 3
6 1
3 3
3 1
0 6
4 4
3 6
6 3
0 3
5 3
0 3
3 3
3 5
3 3
6 6
3 5
0 1
3 1
3 0
3 0
0 4
3 3
3 0
0 0
3 5
5 5
3 6
6 6
6 6
0 0
3 3
6 3
0 0
3 0
3 1
6 0
3 1
0 0
6 6
3 6
0 0
3 6
0 0
3 3
3 5
3 5
0 6
0 3
1 1
4 1
0 1
3 0
3 3